<a href="https://colab.research.google.com/github/EmamulHossen/Spam_Email_Detection/blob/main/Spam_Email_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# **Importing Modules**

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import confusion_matrix,roc_curve,roc_auc_score,auc

**Load data**

In [4]:
data=pd.read_csv("/content/drive/MyDrive/Data/emails.csv")
text_column = 'text'
spam_column = 'spam'

In [5]:
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [6]:
data.shape

(5728, 2)

In [7]:
#Value Counts of spam & Ham
data["spam"].value_counts() # 1=>spam
                            # 0=>Ham

0    4360
1    1368
Name: spam, dtype: int64

# **Step-1: Remove Punctuation & Stopwords**

In [8]:
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [9]:
data.text[0]

"Subject: naturally irresistible your corporate identity  lt is really hard to recollect a company : the  market is full of suqgestions and the information isoverwhelminq ; but a good  catchy logo , stylish statlonery and outstanding website  will make the task much easier .  we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader : it isguite ciear that  without good products , effective business organization and practicable aim it  will be hotat nowadays market ; but we do promise that your marketing efforts  will become much more effective . here is the list of clear  benefits : creativeness : hand - made , original logos , specially done  to reflect your distinctive company image . convenience : logo and stationery  are provided in all formats ; easy - to - use content management system letsyou  change your website content and even its structure . promptness : you  will see logo drafts within three business days . affordability : your  ma

In [10]:
data.text[350]

'Subject: we are the only online pharmacy offering " 100 % satisfaction guarantee " .  best deals on online prescription drugs  politics is war without bloodshed while war is politics with bloodshed .  \' but \' is a fence over which few leap .  with great power comes great responsibility .  each bird loves to hear himself sing .'

**Stop word**
> Stop words are a set of commonly used words in a language. Examples of stop words in English are “a,” “the,” “is,” “are,” etc.



In [11]:
#Checking stop words in English Language
stw=set(stopwords.words("english"))
stw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [12]:
# Not only english language,stopwords has every language.
stopwords.fileids()

['arabic',
 'azerbaijani',
 'basque',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'turkish']

In [13]:
stw=set(stopwords.words('bengali')) # Stopwords for Bangla Language
stw

{'অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [14]:
string.punctuation # Check the Punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
#All word in lower format
"DaTa".lower()

'data'

# **Some of the preprocessing Steps are:-**
1.**Removing punctuations like . , ! $( ) * % @**<br/>
2.**Removing URLs**<br/>
3.**Removing Stop words**<br/>
4.**Lower casing.**<br/>
5.**Tokenization.**<br/>
6.**Stemming.**<br/>
7.**Lemmatization.**

In [16]:
def preprocess_text(text):
    # Remove punctuation
    remove_punc = [char for char in text if char not in string.punctuation]
    clean_words = ''.join(remove_punc) # char joining

    #Remove stopwords
    text = ([word for word in clean_words.split() if word.lower() not in stw]) # stopword = stopwords.words('english')
    return text

In [17]:
data[text_column] = data[text_column].apply(preprocess_text)

In [18]:
data[text_column]

0       [Subject, naturally, irresistible, your, corpo...
1       [Subject, the, stock, trading, gunslinger, fan...
2       [Subject, unbelievable, new, homes, made, easy...
3       [Subject, 4, color, printing, special, request...
4       [Subject, do, not, have, money, get, software,...
                              ...                        
5723    [Subject, re, research, and, development, char...
5724    [Subject, re, receipts, from, visit, jim, than...
5725    [Subject, re, enron, case, study, update, wow,...
5726    [Subject, re, interest, david, please, call, s...
5727    [Subject, news, aurora, 5, 2, update, aurora, ...
Name: text, Length: 5728, dtype: object

# **Step-2: Lemmatization**

In [19]:
lemmatizer=WordNetLemmatizer()


def lemmatize_text(text):
  lemmatized_text=' '.join([lemmatizer.lemmatize(word) for word in text])
  return lemmatized_text


data[text_column] = data[text_column].apply(lemmatize_text)

In [20]:
data[text_column]

0       Subject naturally irresistible your corporate ...
1       Subject the stock trading gunslinger fanny is ...
2       Subject unbelievable new home made easy im wan...
3       Subject 4 color printing special request addit...
4       Subject do not have money get software cd from...
                              ...                        
5723    Subject re research and development charge to ...
5724    Subject re receipt from visit jim thanks again...
5725    Subject re enron case study update wow all on ...
5726    Subject re interest david please call shirley ...
5727    Subject news aurora 5 2 update aurora version ...
Name: text, Length: 5728, dtype: object

# **Step-3: TF-IDF Vectorizer**

In [21]:
vectorizer=TfidfVectorizer()

x=vectorizer.fit_transform(data[text_column])
y=data[spam_column]

In [22]:
x.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.11472821, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

# **Fit Models**

In [23]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# **Performance Evaluation**

In [24]:
models = [
    MultinomialNB(),
    BernoulliNB()
]

for model in models:
    model.fit(x_train, y_train)

    ypred = model.predict(x_test)
    ypred_proba = model.predict_proba(x_test)[:, 1]

    print(f"Model: {type(model).__name__}")
    print('Accuracy Score =',model.score(x_test, y_test))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, ypred))
    print("AUC Score:", roc_auc_score(y_test, ypred_proba))

    print('\n')



Model: MultinomialNB
Accuracy Score = 0.8525305410122164
Confusion Matrix:
[[856   0]
 [169 121]]
AUC Score: 0.9942193038994521


Model: BernoulliNB
Accuracy Score = 0.9825479930191972
Confusion Matrix:
[[846  10]
 [ 10 280]]
AUC Score: 0.9978649693844667




In [25]:
models = [
    MultinomialNB(),
    BernoulliNB()
]

for model in models:
    model.fit(x_train, y_train)

    ypred = model.predict(x_test)
    ypred_proba = model.predict_proba(x_test)[:,1]

    print(f"Model: {type(model).__name__}")
    print('Accuracy Score =',model.score(x_test, y_test))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, ypred))
    print("AUC Score:", roc_auc_score(y_test, ypred_proba))

    print('\n')

Model: MultinomialNB
Accuracy Score = 0.8525305410122164
Confusion Matrix:
[[856   0]
 [169 121]]
AUC Score: 0.9942193038994521


Model: BernoulliNB
Accuracy Score = 0.9825479930191972
Confusion Matrix:
[[846  10]
 [ 10 280]]
AUC Score: 0.9978649693844667




# **Random Text**

Congratulations! You've been selected as one of our lucky winners in our exclusive Vacation Giveaway! You're just one step away from claiming your dream vacation.

Here's what you've won: 5-star accommodation in a tropical paradise, First-class flights for you and a companion, VIP access to the most exclusive attractions, Gourmet dining experiences, To claim your prize, simply click the link below and provide your contact details: must click the link and submit your credit card information.



# **Step-6: Make Predictions on random text**

In [26]:
random_text = input()

preprocessed_text = preprocess_text(random_text) # remove punctuation
lemmatized_text = lemmatize_text(preprocessed_text) # text scaling
text_vector = vectorizer.transform([lemmatized_text])

for model in models:
    prediction = model.predict(text_vector)
    print(f"Model: {type(model).__name__}")
    print("Prediction:", prediction)
    print('\n')

Congratulations! You've been selected as one of our lucky winners in our exclusive Vacation Giveaway! You're just one step away from claiming your dream vacation.  Here's what you've won: 5-star accommodation in a tropical paradise, First-class flights for you and a companion, VIP access to the most exclusive attractions, Gourmet dining experiences, To claim your prize, simply click the link below and provide your contact details: must click the link and submit your credit card information.
Model: MultinomialNB
Prediction: [0]


Model: BernoulliNB
Prediction: [1]




# **Random Text**
Hello All,
I have made a complete MYSQL guide with entire reference materials and usecases and scenarios based questions and answera for data scientist and data analyst interviews.Happy Learning!!

In [28]:
random_text = input()

preprocessed_text = preprocess_text(random_text) # remove punctuation
lemmatized_text = lemmatize_text(preprocessed_text) # text scaling
text_vector = vectorizer.transform([lemmatized_text])

for model in models:
    prediction = model.predict(text_vector)
    print(f"Model: {type(model).__name__}")
    print("Prediction:", prediction)
    print('\n')

Hello All, I have made a complete MYSQL guide with entire reference materials and usecases and scenarios based questions and answera for data scientist and data analyst interviews.Happy Learning!!
Model: MultinomialNB
Prediction: [0]


Model: BernoulliNB
Prediction: [0]




# **Random Text**
Hi Emamul Hossen,

Most writing assessments focus on only the final product. But what if data science could measure and track key aspects of the writing process itself that reliably produces quality writing?

In this competition, you’ll develop a model that can predict overall writing quality based on a large dataset of keystroke logs that capture the writing process.

In [29]:
random_text = input()

preprocessed_text = preprocess_text(random_text) # remove punctuation
lemmatized_text = lemmatize_text(preprocessed_text) # text scaling
text_vector = vectorizer.transform([lemmatized_text])

for model in models:
    prediction = model.predict(text_vector)
    print(f"Model: {type(model).__name__}")
    print("Prediction:", prediction)
    print('\n')

Hi Emamul Hossen,  Most writing assessments focus on only the final product. But what if data science could measure and track key aspects of the writing process itself that reliably produces quality writing?  In this competition, you’ll develop a model that can predict overall writing quality based on a large dataset of keystroke logs that capture the writing process.
Model: MultinomialNB
Prediction: [0]


Model: BernoulliNB
Prediction: [0]


